In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/')

In [2]:
import os
print(os.environ['PATH'])

/Users/ravi.tej/anaconda3/envs/recoenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin


In [3]:
import boto3

# Create a session using your current IAM role
session = boto3.session.Session()
sts_client = session.client('sts')

# Get the caller identity
caller_identity = sts_client.get_caller_identity()
account_id = caller_identity['Account']

print("Your AWS Account ID is:", account_id)

Your AWS Account ID is: 005418323977


In [4]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model

# Initialize clients
ecr_client = boto3.client('ecr')
s3_client = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker')

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [5]:
# Define your variables
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'cambrian'
image_tag = 'latest'
local_docker_image = f'{ecr_repository}:{image_tag}'
ecr_image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}:{image_tag}'
s3_bucket = 'sagemaker-ap-south-1-005418323977'
s3_key = 'feed_artifacts/model.tar.gz'
s3_uri = f's3://{s3_bucket}/{s3_key}'

iam = boto3.client('iam')
role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']

In [6]:
def create_ecr_repository(repo_name):
    try:
        ecr_client.create_repository(repositoryName=repo_name)
        print(f"Repository {repo_name} created.")
    except ecr_client.exceptions.RepositoryAlreadyExistsException:
        print(f"Repository {repo_name} already exists.")

create_ecr_repository(ecr_repository)

Repository cambrian already exists.


In [7]:
vpc_config={
    'SecurityGroupIds': ['sg-05958586370d44105'],  # Replace with your security group ID(s)
    'Subnets': ['subnet-058a29a25975122a4','subnet-0c31b46eec1921be1','subnet-02a038b3b282743a0']  # Replace with your subnet ID(s)
}

In [8]:
model = Model(model_data='s3://sagemaker-ap-south-1-005418323977/feed_artifacts/model.tar.gz', 
              image_uri='005418323977.dkr.ecr.ap-south-1.amazonaws.com/cambrian:latest',
              role='AmazonSageMaker-ExecutionRole-20231030T210397',
             vpc_config=vpc_config)

In [10]:
model.deploy(instance_type='ml.t2.large', 
             initial_instance_count=1,
             endpoint_name='recommendations-feed-endpoint',
             env={'DB_HOST': 'articleembeddingsandclusters.chmaiagjipqy.ap-south-1.rds.amazonaws.com',
                  'DB_PORT': '5432',
                  'DB_NAME': 'article-embeddings-clusters',
                  'DB_USER': 'postgres',
                  'DB_PASSWORD': 'insightML'
                 })

---------------------------------------*

UnexpectedStatusException: Error hosting endpoint recommendations-feed-endpoint: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [7]:
import subprocess

In [8]:
# Get the ECR login password
login_password = subprocess.check_output(
    f"aws ecr get-login-password --region {region}",
    shell=True, text=True
)

# Login to ECR using --password-stdin
login_command = (
    f"docker login --username AWS --password-stdin "
    f"{account_id}.dkr.ecr.{region}.amazonaws.com"
)

proc = subprocess.Popen(
    login_command, 
    shell=True, 
    stdin=subprocess.PIPE, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True
)

# Provide the password to the login command
stdout, stderr = proc.communicate(login_password)

if proc.returncode != 0:
    print(f"Error: {stderr}")
else:
    print(f"Login successful. Output:\n{stdout}")

/bin/sh: aws: command not found


CalledProcessError: Command 'aws ecr get-login-password --region ap-south-1' returned non-zero exit status 127.

In [22]:
# Tagging the Docker image
!docker tag {local_docker_image} {ecr_image_uri}

# Pushing the Docker image
!docker push {ecr_image_uri}

The push refers to repository [005418323977.dkr.ecr.ap-south-1.amazonaws.com/cambrian]

914175f0: Preparing 
65ea4b54: Preparing 
c89a5c2e: Preparing 
b0a7261a: Preparing 
946e114b: Preparing 
76542c61: Preparing 
a09a3cec: Preparing 
914175f0: Pushed   254.1MB/246.6MBAlatest: digest: sha256:74600bf682da5efd8bb777780d1454a097260d7a4f8acbe85cb9dd3081978867 size: 1998


In [10]:
role

'arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397'

In [11]:
env_variables = ['DB_HOST', 'DB_NAME', 'DB_PASSWORD', 'DB_PORT', 'DB_USER']
env_dict = {x: os.environ.get(x) for x in env_variables}

In [13]:
# Create a SageMaker model
sagemaker_model = Model(
    image_uri=ecr_image_uri,
    model_data=s3_uri,
    role=role,
    sagemaker_session=sagemaker.Session(),
    env=env_dict
)

In [15]:
# Deploy the model to an endpoint
predictor = sagemaker_model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    subnets = ['subnet-058a29a25975122a4','subnet-0c31b46eec1921be1','subnet-02a038b3b282743a0'],
    security_group_ids=['sg-05958586370d44105'],
    enable_network_isolation=False
)

---------------------------------------*

UnexpectedStatusException: Error hosting endpoint cambrian-2024-01-10-13-08-29-585: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html